In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyperclip

In [4]:
answer_df = pd.read_excel()
question_df = pd.read_excel()
response_df = pd.read_csv()

In [14]:
#Prompt prefix
# Newest prompt (c3)
Explanation_eng_c3 = """
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Provide your grade as a single integer value without any additional text or formatting.
The text you are grading is an answer to a question, where the problem is stated within the question itself. Your evaluation should be based on the appropriateness of reasoning and evidence. Please consider the following aspects when assigning the grade:

- Coherence of Claim-Reasoning/Evidence Relationship: Assess whether there is a suitable connection established between the claim and the accompanying reasoning/evidence. Verify if the provided reasons or evidence adequately support the claim or subclaims and if the claim aligns with the relevant issue.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""


Explanation_scheme_eng_c3 = """
As an essay grader, your task is to evaluate and assign a grade on a scale from 1 to 5 to the provided Korean text. Please provide your grade as a single integer value without any additional text or formatting. The text you are grading is an answer to a question, where the problem is stated within the question itself. Your evaluation should be based on the "Coherence of Claim-Reasoning/Evidence Relationship."

Coherence of Claim-Reasoning/Evidence Relationship: Assess whether there is a strong and logical connection established between the claim and the accompanying reasoning/evidence. Verify if the provided reasons or evidence effectively support the claim or subclaims, and if the claim aligns well with the relevant issue.

Please use the following grading scheme:

5 points: Exceptionally appropriate reasoning and evidence, highly persuasive.
4 points: Reasoning and evidence are generally appropriate and persuasive.
3 points: Reasoning and evidence are somewhat acceptable but could be improved.
2 points: Many instances where reasoning and evidence are not appropriate or lack persuasiveness.
1 point: Most of the reasoning and evidence are not appropriate, lacking persuasiveness.

Please assign grades objectively, without hesitation in giving high grades like 5 or low grades like 1. Now, carefully review the following question and answer, and assign a grade accordingly:
"""

Explanation_scheme_sw_eng_c3 = """
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Identify the strengths and weaknesses of the following text and provide your grade as a single integer value without any additional text or formatting. 
The text you are grading is an answer to a question. Your evaluation should be based on the appropriateness of reasoning and evidence. Please consider the following aspects when assigning the grade:

- Coherence of Claim-Reasoning/Evidence Relationship: Assess whether there is a suitable connection established between the claim and the accompanying reasoning/evidence. Verify if the provided reasons or evidence adequately support the claim or subclaims and if the claim aligns with the relevant issue.

Please use the following grading scheme:
5 points: Very appropriate reasoning and evidence, highly persuasive.
4 points: Reasoning and evidence are generally appropriate, somewhat persuasive.
3 points: Some of the reasoning and evidence are not appropriate, but generally acceptable.
2 points: Many cases where reasoning and evidence are not appropriate, low persuasiveness.
1 point: Most of the reasoning and evidence are not appropriate, very low persuasiveness.


Now, carefully review the following question and answer, and assign a grade accordingly:
"""

Explanation_scheme_step_and_score_c3_original="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the appropriateness of reasoning and evidence. Please consider the following aspects when assigning the grade:

- Coherence of Claim-Reasoning/Evidence Relationship: Assess whether there is a suitable connection established between the claim and the accompanying reasoning/evidence. Verify if the provided reasons or evidence adequately support the claim or subclaims and if the claim aligns with the relevant issue.

Please use the following grading scheme:
5 points: Very appropriate reasoning and evidence, highly persuasive.
4 points: Reasoning and evidence are generally appropriate, somewhat persuasive.
3 points: Some of the reasoning and evidence are not appropriate, but generally acceptable.
2 points: Many cases where reasoning and evidence are not appropriate, low persuasiveness.
1 point: Most of the reasoning and evidence are not appropriate, very low persuasiveness.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""


Explanation_kor_c3 = """
수필 평가자로서 제공된 한국어 답변에 대해 1점부터 5점까지의 점수를 매기는 역할을 맡았습니다. 대답은 단일 정숫값으로 점수를 표시해 주세요. 추가 설명은 하지 마세요.
평가하는 글은 주어진 질문에 대한 답변입니다. 평가는 이유와 근거의 적절성을 기반으로 해야 합니다. 점수를 매길 때 다음 측면을 고려해 주세요:

주장-이유/근거 관계의 적절성: 주장 또는 주장의 이유를 뒷받침하는 근거가 적절한지 평가해 주세요. 주장이나 주장의 이유를 충분히 지지하는 근거가 제공되었는지, 주장이 관련 문제와 일치하는지 확인해 주세요.

이제 다음 질문과 답변을 주의 깊게 검토하고 점수를 매겨 주세요:
"""

Explanation_scheme_kor_c3 = """
수필 평가자로서 제공된 한국어 답변에 대해 1점부터 5점까지의 점수를 매기는 역할을 맡았습니다. 대답은 단일 정숫값으로 점수를 표시해 주세요. 추가 설명은 하지 마세요.
평가하는 글은 주어진 질문에 대한 답변입니다. 평가는 이유와 근거의 적절성을 기반으로 해야 합니다. 점수를 매길 때 다음 측면을 고려해 주세요:

주장-이유/근거 관계의 적절성: 주장 또는 주장의 이유를 뒷받침하는 근거가 적절한지 평가해 주세요. 주장이나 주장의 이유를 충분히 지지하는 근거가 제공되었는지, 주장이 관련 문제와 일치하는지 확인해 주세요.

다음과 같은 점수 체계를 사용해 주세요:
5점: 이유나 근거가 매우 적절하여 설득력이 높음.
4점: 이유나 근거가 대체로 적절하여 다소 설득력이 있음.
3점: 이유나 근거 가운데 일부가 적절하지 않으나, 대체로 수용할 만함.
2점: 이유나 근거가 적절하지 않은 경우가 많아 설득력이 떨어짐.
1점: 이유나 근거가 대부분 적절하지 않아 설득력이 매우 낮음.

이제 다음 질문과 답변을 주의 깊게 검토하고 점수를 매겨 주세요:
"""

Explanation_scheme_sw_kor_c3 = """
수필 평가자로서 제공된 한국어 답변에 대해 1점부터 5점까지의 점수를 매기는 역할을 맡았습니다. 대답은 답변의 강점 및 약점을 파악하여 추가 설명 없이 단일 정숫값으로 점수를 표시해 주세요.
평가하는 글은 주어진 질문에 대한 답변입니다. 평가는 이유와 근거의 적절성을 기반으로 해야 합니다. 점수를 매길 때 다음 측면을 고려해 주세요:

주장-이유/근거 관계의 적절성: 주장 또는 주장의 이유를 뒷받침하는 근거가 적절한지 평가해 주세요. 주장이나 주장의 이유를 충분히 지지하는 근거가 제공되었는지, 주장이 관련 문제와 일치하는지 확인해 주세요.

다음과 같은 점수 체계를 사용해 주세요:
5점: 이유나 근거가 매우 적절하여 설득력이 높음.
4점: 이유나 근거가 대체로 적절하여 다소 설득력이 있음.
3점: 이유나 근거 가운데 일부가 적절하지 않으나, 대체로 수용할 만함.
2점: 이유나 근거가 적절하지 않은 경우가 많아 설득력이 떨어짐.
1점: 이유나 근거가 대부분 적절하지 않아 설득력이 매우 낮음.

이제 다음 질문과 답변을 주의 깊게 검토하고 점수를 매겨 주세요:
"""

Explanation_scheme_step_kor_c3 = """
수필 평가자로서 제공된 한국어 답변에 대해 1점부터 5점까지의 점수를 매기는 역할을 맡았습니다. 대답은 철저히 단계별로 사고 과정을 검토한 후, 추가 설명 없이 단일 정숫값으로 점수를 표시해 주세요.
평가하는 글은 주어진 질문에 대한 답변입니다. 평가는 이유와 근거의 적절성을 기반으로 해야 합니다. 점수를 매길 때 다음 측면을 고려해 주세요:

주장-이유/근거 관계의 적절성: 주장 또는 주장의 이유를 뒷받침하는 근거가 적절한지 평가해 주세요. 주장이나 주장의 이유를 충분히 지지하는 근거가 제공되었는지, 주장이 관련 문제와 일치하는지 확인해 주세요.

다음과 같은 점수 체계를 사용해 주세요:
5점: 이유나 근거가 매우 적절하여 설득력이 높음.
4점: 이유나 근거가 대체로 적절하여 다소 설득력이 있음.
3점: 이유나 근거 가운데 일부가 적절하지 않으나, 대체로 수용할 만함.
2점: 이유나 근거가 적절하지 않은 경우가 많아 설득력이 떨어짐.
1점: 이유나 근거가 대부분 적절하지 않아 설득력이 매우 낮음.

이제 다음 질문과 답변을 주의 깊게 검토하고 점수를 매겨 주세요:
"""

In [3]:
Explanation_scheme_step_and_score_c1="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the combined criteria of problem comprehension and explanation quality. Consider the following aspects while grading:

- Problem Comprehension: Assess how effectively the text demonstrates understanding of the problem and its relevance. Focus on the introduction and information presented throughout the text.
- Explanation Quality: Evaluate the informativeness of the explanation provided. Ensure that the text offers sufficient information to clarify the problem concisely.

Please use the following grading scheme:
5 points: The text demonstrates a very effective understanding of the problem, closely related to the issue, and provides necessary information.
4 points: The text demonstrates a generally effective understanding of the problem, closely related to the issue, and includes mostly necessary information.
3 points: The text shows an understanding of the problem related to the issue but lacks some information.
2 points: The text's understanding of the problem contains information that is not closely related to the issue.
1 point: The text provides no explanation of the problem or displays a very weak understanding.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""

Explanation_scheme_step_and_score_c2="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the combined criteria of problem comprehension and explanation quality. Consider the following aspects while grading:

- Problem Comprehension: Assess how effectively the text demonstrates understanding of the problem and its relevance. Focus on the introduction and information presented throughout the text.
- Explanation Quality: Evaluate the informativeness of the explanation provided. Ensure that the text offers sufficient information to clarify the problem concisely.

Please use the following grading scheme:
5 points: The text demonstrates a very effective understanding of the problem, closely related to the issue, and provides necessary information.
4 points: The text demonstrates a generally effective understanding of the problem, closely related to the issue, and includes mostly necessary information.
3 points: The text shows an understanding of the problem related to the issue but lacks some information.
2 points: The text's understanding of the problem contains information that is not closely related to the issue.
1 point: The text provides no explanation of the problem or displays a very weak understanding.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""

Explanation_scheme_step_and_score_c3="""
As an essay grader, your task is to evaluate and assign a grade on the scale of 1 to 5 on the provided Korean essay. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The essay you are grading is about a discussion point. Your evaluation should be based on the appropriateness of reasoning and evidence. Please consider the following aspects when assigning the grade:

- Coherence of Claim-Reasoning/Evidence Relationship: Assess whether there is a suitable connection established between the claim and the accompanying reasoning/evidence. Verify if the provided reasons or evidence adequately support the claim or subclaims and if the claim aligns with the relevant issue.

Please use the following grading scheme:
5 points: Very appropriate reasoning and evidence, highly persuasive.
4 points: Reasoning and evidence are generally appropriate, somewhat persuasive.
3 points: Some of the reasoning and evidence are not appropriate, but generally acceptable.
2 points: Many cases where reasoning and evidence are not appropriate, low persuasiveness.
1 point: Most of the reasoning and evidence are not appropriate, very low persuasiveness.

Now, carefully review the following discussion point and essay, and assign a grade accordingly:
"""

Explanation_scheme_step_and_score_c4="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the appropriateness of reasoning and evidence. Please consider the following aspects when assigning the grade:

- Coherence of Claim-Reasoning/Evidence Relationship: Assess whether there is a suitable connection established between the claim and the accompanying reasoning/evidence. Verify if the provided reasons or evidence adequately support the claim or subclaims and if the claim aligns with the relevant issue.

Please use the following grading scheme:
5 points: Very appropriate reasoning and evidence, highly persuasive.
4 points: Reasoning and evidence are generally appropriate, somewhat persuasive.
3 points: Some of the reasoning and evidence are not appropriate, but generally acceptable.
2 points: Many cases where reasoning and evidence are not appropriate, low persuasiveness.
1 point: Most of the reasoning and evidence are not appropriate, very low persuasiveness.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""

Explanation_scheme_step_and_score_c5="""
As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Re-evaluate your thought process step by step to show how it reflects the grading scheme. Provide your grade as a single integer value.
The text you are grading is an answer to a question. Your evaluation should be based on the appropriateness of reasoning and evidence. Please consider the following aspects when assigning the grade:

- Coherence of Claim-Reasoning/Evidence Relationship: Assess whether there is a suitable connection established between the claim and the accompanying reasoning/evidence. Verify if the provided reasons or evidence adequately support the claim or subclaims and if the claim aligns with the relevant issue.

Please use the following grading scheme:
5 points: Very appropriate reasoning and evidence, highly persuasive.
4 points: Reasoning and evidence are generally appropriate, somewhat persuasive.
3 points: Some of the reasoning and evidence are not appropriate, but generally acceptable.
2 points: Many cases where reasoning and evidence are not appropriate, low persuasiveness.
1 point: Most of the reasoning and evidence are not appropriate, very low persuasiveness.

Now, carefully review the following question and answer, and assign a grade accordingly:
"""

In [8]:
def print_combined_text(prompt_type, text_id):
    text_column = answer_df[answer_df['EXAMINEE_ID'] == text_id]['TEXT'].values[0]
    question_column = question_df[question_df['Q_NUM'] == int(text_id[1])]['Q_TEXT'].values
    combined_text = f"{prompt_type}\n**Discussion point:**\n {question_column}\n\n**Essay to grade:**\n [{text_column}]"
    print(combined_text)
    pyperclip.copy(combined_text)
    print("Copied to clipboard!")

In [54]:
print_combined_text(exo, "A1-01")


As an essay grader, your task is to evaluate and assign a grade between 1 and 5 to the provided Korean text. Provide your grade as a single integer value without any additional text or formatting.
The text you are grading is an answer to a question. Your evaluation should be based on the appropriateness of reasoning and evidence. Please consider the following aspects when assigning the grade:

- Coherence of Claim-Reasoning/Evidence Relationship: Assess whether there is a suitable connection established between the claim and the accompanying reasoning/evidence. Verify if the provided reasons or evidence adequately support the claim or subclaims and if the claim aligns with the relevant issue.

Please use the following grading scheme:
5 points: Very appropriate reasoning and evidence, highly persuasive.
4 points: Reasoning and evidence are generally appropriate, somewhat persuasive.
3 points: Some of the reasoning and evidence are not appropriate, but generally acceptable.
2 points: Ma

In [3]:
# 질문답변 csv화
def generate_promptqa(examinee_id, question_text, text_to_grade):
    return pd.Series([examinee_id, f"**질문:**\n {question_text}\n\n**채점할 답변:**\n [{text_to_grade}]"])

def generate_prompts(data):
    prompts = []
    for index, row in data.iterrows():
        examinee_id = row['EXAMINEE_ID']
        text_column = row['TEXT']
        question_column = question_df[question_df['Q_NUM'] == int(examinee_id[1])]['Q_TEXT'].values[0]
        prompt = generate_promptqa(examinee_id, question_column, text_column)
        prompts.append(prompt)
    
    return pd.concat(prompts, axis=1).T

prompts_df = generate_prompts(answer_df)  # Pass the correct DataFrame object
prompts_df.to_csv(r'', index = False)

In [7]:
import json

def generate_prompt(essay_prompt, essay, grade):
    return {
        "instruction": essay_prompt,
        "input": essay,
        "output": grade
    }

def generate_prompts(json_file_path):
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    prompts = []
    for entry in data:
        question_number = entry['Q']
        essay = entry['TEXT']
        grade = entry['C1']

        # Get the essay prompt from the Q variable using the question_number
        essay_prompt = question_df[question_df['Q_NUM'] == question_number]['Q_TEXT'].values[0]

        prompt = generate_prompt(essay_prompt, essay, grade)
        prompts.append(prompt)

    return prompts


# Replace 'path_to_your_json_file.json' with the actual file path
json_file_path = r""

# Call your function and get the prompts list
prompts_list = generate_prompts(json_file_path)

# Save the prompts list as a JSON file
with open(r'', 'w', encoding='utf-8') as json_file:
    json.dump(prompts_list, json_file, ensure_ascii=False, indent=4)


In [6]:
# USing GPT response for grade
import pandas as pd
import json

def generate_prompt(essay_prompt, essay, grade):
    return {
        "instruction": essay_prompt,
        "input": essay,
        "output": grade
    }

def generate_prompts(json_file_path, excel_file_path):
    # Read the C1_gpt_json.xlsx file and set '글 자료 ID' as the index
    excel_data = pd.read_excel(excel_file_path, index_col='글 자료 ID')

    # Read the original JSON file
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        data = json.load(json_file)

    prompts = []
    for entry in data:
        EXAMINEE_ID = entry["EXAMINEE_ID"]
        question_number = entry['Q']

        # Check if the '글 자료 ID' exists in the C1_gpt_json.xlsx file
        if EXAMINEE_ID in excel_data.index:
            # Get the essay prompt from the Q variable using the question_number
            essay_prompt = question_df[question_df['Q_NUM'] == question_number]['Q_TEXT'].values[0]
            grade = excel_data.loc[EXAMINEE_ID, 'C1_responses_gpt3.5']
            essay = entry['TEXT']

            prompt = generate_prompt(essay_prompt, essay, grade)
            prompts.append(prompt)

    return prompts

# Replace 'path_to_your_json_file.json' with the actual file path
json_file_path = r""
# Replace 'path_to_your_excel_file.xlsx' with the actual file path
excel_file_path = r""

# Call your function and get the prompts list
prompts_list = generate_prompts(json_file_path, excel_file_path)

# Save the prompts list as a JSON file
with open(r'', 'w', encoding='utf-8') as json_file:
    json.dump(prompts_list, json_file, ensure_ascii=False, indent=4)


In [6]:
# USing GPT response for grade
import pandas as pd
import json
excel_file_path = r""
def translatejson(response, grade):
    return {
        "Response": response,
        "Grade": grade,
    }


excel_data = pd.read_excel(excel_file_path)
grade = excel_data["Grade"].tolist()
explanation = excel_data["Explanation"].tolist()
prompts = []
for i, j in zip(explanation,grade):  
    prompts.append(translatejson(i,j))


# Save the prompts list as a JSON file
with open(r'', 'w', encoding='utf-8') as json_file:
    json.dump(prompts, json_file, ensure_ascii=False, indent=4)


In [2]:
excel_data

,C1_responses_gpt3.5,Grade,Explanation
0,Grade: 3\n\nExplanation:\nThe answer demonstra...,3,\n\nExplanation:\nThe answer demonstrates a ge...
1,Grade: 3\n\nExplanation:\nThe answer demonstra...,3,\n\nExplanation:\nThe answer demonstrates a ge...
2,Grade: 4\n\nExplanation:\nThe answer demonstra...,4,\n\nExplanation:\nThe answer demonstrates a ge...
3,Grade: 3\n\nExplanation:\nThe answer demonstra...,3,\n\nExplanation:\nThe answer demonstrates a ge...
4,Grade: 4\n\nExplanation:\nThe answer demonstra...,4,\n\nExplanation:\nThe answer demonstrates a ge...
...,...,...,...
296,Grade: 4\n\nExplanation:\nThe answer demonstra...,4,\n\nExplanation:\nThe answer demonstrates a ge...
297,Grade: 4\n\nExplanation:\nThe answer demonstra...,4,\n\nExplanation:\nThe answer demonstrates a ge...
298,Grade: 4\n\nExplanation:\nThe answer demonstra...,4,\n\nExplanation:\nThe answer demonstrates a ge...
299,Grade: 4\n\nExplanation:\nThe answer demonstra...,4,\n\nExplanation:\nThe answer demonstrates a ge...
